# Egobox

## Installation

You can install with the `pip` command as follows: 

```bash
pip install egobox
```

## Import

In [1]:
import numpy as np
import egobox as egx

# To display optimization information (none by default)
import logging
logging.basicConfig(level=logging.INFO)

## Continuous test functions 

In [2]:
xspecs_xsinx = egx.to_specs([[0., 25.]])
n_cstr_xsinx = 0

def xsinx(x: np.ndarray) -> np.ndarray:
    x = np.atleast_2d(x)
    y = (x - 3.5) * np.sin((x - 3.5) / (np.pi))
    return y

In [3]:
xspecs_g24 = egx.to_specs([[0., 3.], [0., 4.]])
n_cstr_g24 = 2

# Objective
def G24(point):
    """
    Function g24
    1 global optimum y_opt = -5.5080 at x_opt =(2.3295, 3.1785)
    """
    p = np.atleast_2d(point)
    return - p[:, 0] - p[:, 1]

# Constraints < 0
def G24_c1(point):
    p = np.atleast_2d(point)
    return (- 2.0 * p[:, 0] ** 4.0
            + 8.0 * p[:, 0] ** 3.0 
            - 8.0 * p[:, 0] ** 2.0 
            + p[:, 1] - 2.0)

def G24_c2(point):
    p = np.atleast_2d(point)
    return (-4.0 * p[:, 0] ** 4.0
            + 32.0 * p[:, 0] ** 3.0
            - 88.0 * p[:, 0] ** 2.0
            + 96.0 * p[:, 0]
            + p[:, 1] - 36.0)

# Grouped evaluation
def g24(point):
    p = np.atleast_2d(point)
    return np.array([G24(p), G24_c1(p), G24_c2(p)]).T


## Ego optimizer written in Rust (a.k.a Egor)

In [4]:
egor = egx.Egor(g24, xspecs_g24, 
                     n_doe=10, 
                     n_cstr=n_cstr_g24, 
                     cstr_tol=1e-3,
                     infill_strategy=egx.InfillStrategy.WB2,
                     # expected=egx.ExpectedOptimum(val=-5.50, tol=1e-2),
                     # outdir="./out",
                     # hot_start=True
                    )  # see help(egor) for options

# Restrict regression and correlation models used
#egor = egx.Egor(g24, xlimits_g24, n_cstr=n_cstr_g24, n_doe=10,
#                      regr_spec=egx.RegressionSpec.LINEAR,
#                      corr_spec=egx.CorrelationSpec.MATERN32 | egx.CorrelationSpec.MATERN52)  

In [5]:
res = egor.minimize(n_iter=30)
print(f"Optimization f={res.y_opt} at {res.x_opt}")

INFO:egobox_ego.egor_solver:Compute initial LHS with 10 points
INFO:egobox_ego.egor_solver:Train surrogates with 10 points...
INFO:egobox_ego.egor_solver:Objective initial clustering and training...
INFO:egobox_ego.egor_solver:... Objective trained (1 / Mixture[Smooth(1)])
INFO:egobox_ego.egor_solver:Constraint[1] initial clustering and training...
INFO:egobox_ego.egor_solver:Constraint[2] initial clustering and training...
INFO:egobox_ego.egor_solver:... Constraint[1] trained (1 / Mixture[Smooth(1)])
INFO:egobox_ego.egor_solver:... Constraint[2] trained (1 / Mixture[Smooth(1)])
INFO:egobox_ego.egor_solver:Infill criterion scaling is updated to 6.794193715904692
INFO:egobox_ego.egor_solver:Constraints scaling is updated to [13.946405245974248, 32.517704189409955]
INFO:egobox_ego.egor_solver:Optimize infill criterion...
INFO:egobox_ego.egor_solver:+1 point(s), total: 11 points
INFO:egobox_ego.egor_solver:********* End iteration 1/30: Best fun(x)=[-4.061137146646022, -2.00661658233285, -

Optimization f=[-5.50864570218447, -4.304372380481425e-05, 0.0009051183207233748] at [2.3295938922233526, 3.179051809961118]


## Mixed integer test function

In [6]:
xspecs_mixint_xsinx = [egx.Vspec(egx.Vtype(egx.Vtype.INT), [0, 25])]
n_cstr_mixint_xsinx = 0

def mixint_xsinx(x: np.ndarray) -> np.ndarray:
    x = np.atleast_2d(x)
    if (np.abs(np.linalg.norm(np.floor(x))-np.linalg.norm(x))< 1e-8):
        y = (x - 3.5) * np.sin((x - 3.5) / (np.pi))
    else:
        raise ValueError(f"Bad input: mixint_xsinx accepts integer only, got {x}")
    return y

In [7]:
egor = egx.Egor(mixint_xsinx, xspecs_mixint_xsinx, 
                     n_doe=3, 
                     infill_strategy=egx.InfillStrategy.EI,
                     target=-15.12,
                    )  # see help(egor) for options
res = egor.minimize(n_iter=30)
print(f"Optimization f={res.y_opt} at {res.x_opt}")

INFO:egobox_ego.egor_solver:Compute initial LHS with 3 points
INFO:egobox_ego.egor_solver:History: 
[[19, -15.121611536143384],
 [14, -2.092831226407067],
 [7, 3.14127615863859]]
INFO:egobox_ego.egor_solver:Optim Result: min f(x)=[-15.121611536143384] at x=[19]


Optimization f=[-15.121611536143384] at [19.0]


## Usage

In [8]:
help(egor)

Help on Egor in module builtins object:

class Egor(object)
 |  Egor(fun, n_cstr=0, cstr_tol=1e-06, n_start=20, n_doe=0, regression_spec=7, correlation_spec=15, infill_strategy=1, q_points=1, par_infill_strategy=1, infill_optimizer=1, n_clusters=1)
 |  
 |  Optimizer constructor
 |  
 |  fun: array[n, nx]) -> array[n, ny]
 |       the function to be minimized
 |       fun(x) = [obj(x), cstr_1(x), ... cstr_k(x)] where
 |          obj is the objective function [n, nx] -> [n, 1]
 |          cstr_i is the ith constraint function [n, nx] -> [n, 1]
 |          an k the number of constraints (n_cstr)
 |          hence ny = 1 (obj) + k (cstrs)
 |       cstr functions are expected be negative (<=0) at the optimum.
 |  
 |   n_cstr (int):
 |       the number of constraint functions.
 |  
 |   cstr_tol (float):
 |       tolerance on constraints violation (cstr < tol).
 |  
 |   xspecs (list(Vspec)) where Vspec(vtype=FLOAT|INT, vlimits=[lower bound, upper bound]):
 |       Bounds of the nx compone